Write out .vtk files of slip vectors on selected faults for viewing in paraview

In [1]:
# Import relevant modules
from rsqsim_api.catalogue.catalogue import RsqSimCatalogue
from rsqsim_api.fault.multifault import RsqSimMultiFault
import fnmatch
import os
from matplotlib import pyplot as plt
import numpy as np

# # Tell python where field paths etc are relative to
script_dir = os.path.abspath('')
fault_dir = "../../../data/subduction"
catalogue_dir = fault_dir


Read in faults and earthquake catalogue. The files are quite large and can be downloaded from https://doi.org/10.5281/zenodo.5534462. Using subduction catalogue and fault model only saves RAM.

In [2]:
fault_model = RsqSimMultiFault.read_fault_file_keith(os.path.join(fault_dir,"hik_creep.flt"))
short_catalogue = RsqSimCatalogue.from_csv_and_arrays(os.path.join(catalogue_dir,"trimmed"))

Select faults of interest (N.B. short catalogue only contains 'subduction'). Use fault_model.names to access list of possible fault names

In [3]:
main_fault="subd"
faults2select = [name for name in fault_model.names if fnmatch.fnmatch(name, main_fault+"*")]
fault_selection=RsqSimMultiFault([fault_model.name_dic[name] for name in faults2select])

Set horizontal azimuth of slip vector for fault.
Slip vector azimuths aren't in the CFM so input desired slip vector manually for now.

In [4]:
slipvec=90.

Calculate rake on each patch to maintain this azimuth of slip vector.

In [5]:
for patchNo in fault_selection.patch_dic:
    patch=fault_selection.patch_dic[patchNo]
    patch.rake=patch.horizontal_sv_to_ds_ss(slipvec)[2]

Write out .vtk file with slip and rake on selected faults to be viewed in Paraview.

In [6]:
fault_selection.slip_rate_to_vtk(vtk_file=main_fault+"_cst_sv.vtk",min_slip_rate=None,nztm_to_lonlat = False)

In [7]:
for patchNo in fault_selection.patch_dic:
    patch=fault_selection.patch_dic[patchNo]
    patch.rake_from_stress_tensor(sigma1=np.array([0,1,0]))

In [8]:
fault_selection.slip_rate_to_vtk(vtk_file=main_fault+"_sigma1.vtk",min_slip_rate=None,nztm_to_lonlat = False)